In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("MySQL to Spark") \
    .config("spark.driver.extraClassPath", "mysql-connector-j-8.0.33.jar") \
    .getOrCreate()

url = "jdbc:mysql://192.168.23.22:3306/bi_data_analyst"
properties = {
    "user": "traht",
    "password": "qCJtXpUrhVPca9WZ5n47",
    "driver": "com.mysql.jdbc.Driver" 
}



Picked up _JAVA_OPTIONS: -Djava.io.tmpdir=/data/jupyterhub/tmp
Picked up _JAVA_OPTIONS: -Djava.io.tmpdir=/data/jupyterhub/tmp
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/21 17:53:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
query = """(SELECT 
            date, total_user, new_user, continue_user, pageview, site, type, session, duration, to_post, to_cate 
            FROM website_overview) 
            AS subquery"""
df_overview = spark.read.jdbc(url=url, table=query, properties=properties)

df_overview.show()


Loading class `com.mysql.jdbc.Driver'. This is deprecated. The new driver class is `com.mysql.cj.jdbc.Driver'. The driver is automatically registered via the SPI and manual loading of the driver class is generally unnecessary.


+-------------------+----------+--------+-------------+--------+------+------+-------+--------+-------+-------+
|               date|total_user|new_user|continue_user|pageview|  site|  type|session|duration|to_post|to_cate|
+-------------------+----------+--------+-------------+--------+------+------+-------+--------+-------+-------+
|2021-11-05 00:00:00|    230110|  116594|        39696|  350631|  genk|mobile| 245594|      63|   NULL|   NULL|
|2021-11-05 00:00:00|     47741|   13261|        16396|  107500|  genk|    pc|  98102|     137|   NULL|   NULL|
|2021-11-02 00:00:00|     44136|   12024|        16437|  105282|  genk|    pc|  91768|     146|   NULL|   NULL|
|2021-11-03 00:00:00|    322465|  177895|        46426|  448031|  genk|mobile| 336276|      56|   NULL|   NULL|
|2021-11-02 00:00:00|    159782|   86123|        29435|  263225|  genk|mobile| 163083|      73|   NULL|   NULL|
|2021-11-03 00:00:00|     55461|   16746|        17071|  115814|  genk|    pc| 107415|     127|   NULL| 

In [4]:
#Overview theo ngày
from pyspark.sql.functions import *

df_overview = df_overview.withColumn('new_user_pct', col('new_user') / col('total_user'))\
                        .withColumn('continue_user_pct', col('continue_user') / col('total_user'))\
                        .withColumn('duration_mins', col('duration') / 60)\
                        .withColumn('page_per_session', col('pageview') / col('session'))

result_df = df_overview.groupBy('date', 'site').agg(
    {'total_user': 'avg',
     'new_user_pct': 'avg',
     'continue_user_pct': 'avg',
     'duration_mins': 'avg',
     'page_per_session': 'avg',
     'session': 'avg',
     'pageview': 'avg'})

# Hiển thị kết quả
result_df = result_df.orderBy('date')
result_df.show()


+-------------------+-------+-------------------+---------------+------------------+----------------------+---------------------+------------+-------------+
|               date|   site|  avg(new_user_pct)|avg(total_user)|avg(duration_mins)|avg(continue_user_pct)|avg(page_per_session)|avg(session)|avg(pageview)|
+-------------------+-------+-------------------+---------------+------------------+----------------------+---------------------+------------+-------------+
|2021-09-30 00:00:00|afamily|0.33297254795519104|       383121.0| 3.408333333333333|    0.3765062180160217|   2.7522405882498404|    484888.0|    1087940.0|
|2021-09-30 00:00:00|cafebiz|0.35476807594067605|       198767.0|              2.15|    0.3113360167347009|    1.876944933273877|    229676.0|     353648.5|
|2021-10-01 00:00:00|afamily| 0.3448233800960743|       339037.0|3.5166666666666666|   0.37917665150928115|     2.82892906029468|    436593.0|    1018885.5|
|2021-10-01 00:00:00|cafebiz| 0.3327040294301497|       17

In [5]:
#Hành động của user sau khi vào home

from pyspark.sql.functions import *

# Lọc dữ liệu để giữ lại các hàng mà có giá trị không phải NaN trong cột 'to_post' hoặc 'to_cate'
filtered_df = df_overview.filter(col('to_post').isNotNull() | col('to_cate').isNotNull())

# Nhóm dữ liệu theo cột 'date' và tính tổng các giá trị của cột 'to_post' và 'to_cate'
grouped_df = filtered_df.groupBy('date').agg(
    {'to_post':'sum',
    'to_cate':'sum'})

grouped_df_sorted = grouped_df.orderBy('date')

grouped_df_sorted.show()



+-------------------+------------+------------+
|               date|sum(to_cate)|sum(to_post)|
+-------------------+------------+------------+
|2021-11-07 00:00:00|      211793|     2633487|
|2021-11-08 00:00:00|      345417|     3134895|
|2021-11-09 00:00:00|      327180|     3787850|
|2021-11-10 00:00:00|      319941|     3652674|
|2021-11-11 00:00:00|      315802|     2946177|
|2021-11-12 00:00:00|      316640|     3127124|
|2021-11-13 00:00:00|      213333|     2602316|
|2021-11-14 00:00:00|      205537|     2690300|
|2021-11-15 00:00:00|      332834|     3214854|
|2021-11-16 00:00:00|      324891|     3240776|
|2021-11-17 00:00:00|      308997|     2906340|
|2021-11-18 00:00:00|      337807|     3177401|
|2021-11-19 00:00:00|      326180|     2904908|
|2021-11-20 00:00:00|      215874|     2541697|
|2021-11-21 00:00:00|      200250|     2511205|
|2021-11-22 00:00:00|      319545|     3305877|
|2021-11-23 00:00:00|      311693|     3515077|
|2021-11-24 00:00:00|      320073|     3

In [6]:
#[Kết quả không hợp lý*******]

from pyspark.sql.functions import col, expr, date_format, lit, concat, format_number, year, weekofyear

# Chuyển đổi cột 'date' thành kiểu timestamp
df_overview = df_overview.withColumn("date", col("date").cast("timestamp"))

# Tạo cột 'week' theo định dạng tuần/năm, không có phần thập phân
df_overview = df_overview.withColumn("week", 
    concat(
        format_number(weekofyear(col("date")), 0),  # Đảm bảo không có phần thập phân
        lit('/'),
        year(col("date"))
    )
)

# Tính toán 'start_date' và 'end_date'
df_overview = df_overview.withColumn("start_date", expr("date_sub(date, dayofweek(date))"))
df_overview = df_overview.withColumn("end_date", expr("date_add(start_date, 6)"))

# Tạo cột 'date_range'
df_overview = df_overview.withColumn("date_range", 
    concat(
        date_format(col("start_date"), 'dd/MM'),
        lit('-'),
        date_format(col("end_date"), 'dd/MM')
    )
)

# Tính toán 'duration_mins' và 'page_per_session'
df_overview = df_overview.withColumn("duration_mins", col("duration") / 60)
df_overview = df_overview.withColumn("page_per_session", 
    when(col("session") > 0, col("pageview") / col("session")).otherwise(lit(None))
)

# Nhóm dữ liệu theo các cột và tính trung bình
group_df = df_overview.groupBy('week', 'date','date_range', 'start_date').agg(
    {'session': 'avg',
     'pageview': 'avg',
     'duration_mins': 'avg',
     'page_per_session': 'avg'}
)

# Đổi tên các cột
group_df = group_df.withColumnRenamed('avg(session)', 'session') \
                   .withColumnRenamed('avg(pageview)', 'pageview') \
                   .withColumnRenamed('avg(duration_mins)', 'duration_mins') \
                   .withColumnRenamed('avg(page_per_session)', 'page_per_session')

# Sắp xếp theo cột 'start_date'
group_df_sorted = group_df.orderBy('start_date')

# Hiển thị kết quả
group_df_sorted.show(truncate=False)


+-------+-------------------+-----------+----------+------------------+------------------+------------------+------------------+
|week   |date               |date_range |start_date|duration_mins     |session           |page_per_session  |pageview          |
+-------+-------------------+-----------+----------+------------------+------------------+------------------+------------------+
|39/2021|2021-10-01 00:00:00|25/09-01/10|2021-09-25|2.8749999999999996|319048.0          |2.3964154234198825|670332.75         |
|39/2021|2021-10-02 00:00:00|25/09-01/10|2021-09-25|2.729166666666667 |291990.5          |2.3462248409767286|609716.0          |
|39/2021|2021-09-30 00:00:00|25/09-01/10|2021-09-25|2.779166666666667 |357282.0          |2.3145927607618586|720794.25         |
|39/2021|2021-10-03 00:00:00|02/10-08/10|2021-10-02|2.5708333333333333|290665.75         |2.287257107394535 |595292.75         |
|40/2021|2021-10-07 00:00:00|02/10-08/10|2021-10-02|3.0583333333333336|324604.25         |2.42799

In [7]:
#Import website_news_info

query = """(SELECT 
            newsid,sourceNews, url, title, publish_date,publish_hour,publish_week,category
            FROM  website_news_info) 
            AS subquery"""
df_info = spark.read.jdbc(url=url, table=query, properties=properties)

df_info.show()


+------------------+----------+--------------------+--------------------+-------------------+------------+------------+----------+
|            newsid|sourceNews|                 url|               title|       publish_date|publish_hour|publish_week|  category|
+------------------+----------+--------------------+--------------------+-------------------+------------+------------+----------+
|176231219171932713|   cafebiz|https://cafebiz.v...|Lười vứt rác, đam...|2024-04-14 00:00:00|          15|     15/2024|      Sống|
|176231222230803748|   cafebiz|https://cafebiz.v...|Hoa khôi đại học ...|2024-02-23 00:00:00|          16|     08/2024|      Sống|
|176231228214247919|   cafebiz|https://cafebiz.v...|Nuôi con giàu và ...|2024-03-31 00:00:00|          15|     13/2024|      Sống|
|176240219145418717|   cafebiz|https://cafebiz.v...|Đàn ông sau 35 tu...|2024-03-30 00:00:00|          16|     13/2024|      Sống|
|176240219152001203|   cafebiz|https://cafebiz.v...|35 tuổi là một cộ...|2024-03-19

In [8]:
#Sản lượng post theo ngày

from pyspark.sql.functions import countDistinct
group_df=df_info.groupBy('publish_date').agg(countDistinct('newsid').alias('newsid')).orderBy('publish_date')
group_df.show()


+-------------------+------+
|       publish_date|newsid|
+-------------------+------+
|2024-02-23 00:00:00|  1244|
|2024-02-24 00:00:00|  1036|
|2024-02-25 00:00:00|   977|
|2024-02-26 00:00:00|  1293|
|2024-02-27 00:00:00|  1335|
|2024-02-28 00:00:00|  1332|
|2024-02-29 00:00:00|  1315|
|2024-03-01 00:00:00|  1303|
|2024-03-02 00:00:00|  1039|
|2024-03-03 00:00:00|   973|
|2024-03-04 00:00:00|  1275|
|2024-03-05 00:00:00|  1345|
|2024-03-06 00:00:00|  1388|
|2024-03-07 00:00:00|  1327|
|2024-03-08 00:00:00|  1311|
|2024-03-09 00:00:00|  1091|
|2024-03-10 00:00:00|   972|
|2024-03-11 00:00:00|  1333|
|2024-03-12 00:00:00|  1292|
|2024-03-13 00:00:00|  1324|
+-------------------+------+
only showing top 20 rows



In [9]:
# #Sản lượng toàn kênh theo tuần

# from pyspark.sql.functions import col, date_sub, date_add, countDistinct, dayofweek, expr

# df_info = df_info.withColumn("publish_date", col("publish_date").cast("timestamp"))

# # Tính ngày đầu tuần (start_date)
# df_info = df_info.withColumn(
#     "start_date",
#     date_sub(col("publish_date"), dayofweek(col("publish_date")) - 1)
# )

# # Tính ngày cuối tuần (end_date) bằng cách cộng thêm 6 ngày vào start_date
# df_info = df_info.withColumn(
#     "end_date",
#     date_add(col("start_date"), 6)
# )

# # Tính tổng số unique newsid theo tuần
# group_df = df_info.groupBy('publish_week', 'start_date', 'end_date').agg(
#     countDistinct('newsid').alias('newsid')
# ).orderBy(['start_date', 'newsid'], ascending=[True, False])

# group_df.show(truncate=False)



In [10]:
#Lượng bài viết suất bản theo chuyên mục

from pyspark.sql.functions import *
group_df=df_info.groupBy('publish_date','category').agg(countDistinct('newsid').alias('newsid')).orderBy(['publish_date','newsid'],ascending=[True,False])
group_df.show()

+-------------------+--------------------+------+
|       publish_date|            category|newsid|
+-------------------+--------------------+------+
|2024-02-23 00:00:00|                Sống|   125|
|2024-02-23 00:00:00|              Xã hội|   101|
|2024-02-23 00:00:00|          Kinh doanh|    84|
|2024-02-23 00:00:00|            Đời sống|    50|
|2024-02-23 00:00:00|            Giải trí|    46|
|2024-02-23 00:00:00|           Lifestyle|    46|
|2024-02-23 00:00:00|   Tài chính quốc tế|    37|
|2024-02-23 00:00:00|       Nhịp sống mới|    35|
|2024-02-23 00:00:00|    Thời sự - Xã hội|    34|
|2024-02-23 00:00:00|                Star|    34|
|2024-02-23 00:00:00|            Thể thao|    33|
|2024-02-23 00:00:00|            Sức khỏe|    30|
|2024-02-23 00:00:00|             Quốc tế|    30|
|2024-02-23 00:00:00|          Thị trường|    28|
|2024-02-23 00:00:00|           Sống khỏe|    22|
|2024-02-23 00:00:00|               Sport|    21|
|2024-02-23 00:00:00|Kinh tế vĩ mô - Đ...|    21|


In [11]:
#Import website_news_info

query = """(SELECT 
            date, refer, landing_page, type, site, access, new_access
            FROM  website_traffic_source) 
            AS subquery"""
df_source = spark.read.jdbc(url=url, table=query, properties=properties)

df_source.show()


+-------------------+----------+------------+------+----+------+----------+
|               date|     refer|landing_page|  type|site|access|new_access|
+-------------------+----------+------------+------+----+------+----------+
|2022-07-13 00:00:00|   statics|      detail|    pc|genk|     1|      NULL|
|2022-07-13 00:00:00|  helo-app|    category|mobile|genk|    17|        16|
|2022-07-13 00:00:00|    google|        home|mobile|genk|   763|       396|
|2022-07-13 00:00:00|      genk|        home|mobile|genk|   335|       138|
|2022-07-13 00:00:00|     adela|      detail|    pc|genk|     2|      NULL|
|2022-07-13 00:00:00|     touch|    category|mobile|genk|    96|        29|
|2022-07-13 00:00:00|     touch|      detail|mobile|genk|    74|        56|
|2022-07-13 00:00:00|     touch|      detail|    pc|genk|     5|         3|
|2022-07-13 00:00:00|     laban|        home|mobile|genk|    10|         3|
|2022-07-13 00:00:00|      bing|    category|mobile|genk|     3|         3|
|2022-07-13 

In [12]:
#Lượng truy cập theo landingpage

from pyspark.sql.functions import *
group_source = df_source.groupBy('date', 'landing_page').agg(sum('access').alias('total_access')).orderBy(['date', 'total_access'], ascending=[True, False])
group_source.show()

+-------------------+------------+------------+
|               date|landing_page|total_access|
+-------------------+------------+------------+
|2022-07-13 00:00:00|      detail|     1652375|
|2022-07-13 00:00:00|        home|      922437|
|2022-07-13 00:00:00|    category|      492154|
|2022-07-13 00:00:00|         tag|       18909|
|2022-07-14 00:00:00|      detail|     1796233|
|2022-07-14 00:00:00|        home|      902007|
|2022-07-14 00:00:00|    category|      518287|
|2022-07-14 00:00:00|         tag|       18940|
|2022-07-15 00:00:00|      detail|     1797836|
|2022-07-15 00:00:00|        home|      907339|
|2022-07-15 00:00:00|    category|      478676|
|2022-07-15 00:00:00|         tag|       19268|
|2022-07-16 00:00:00|      detail|     1691417|
|2022-07-16 00:00:00|        home|      804443|
|2022-07-16 00:00:00|    category|      408097|
|2022-07-16 00:00:00|         tag|       17105|
|2022-07-17 00:00:00|      detail|     1635323|
|2022-07-17 00:00:00|        home|      

In [13]:
#Top 10 traffic source có lượng truy cập cao nhất

from pyspark.sql.functions import *
group_source=df_source.groupBy('refer').agg(sum('access').alias('total_access')).orderBy('total_access',ascending=False)
group_source.show(10)

+-----------+------------+
|      refer|total_access|
+-----------+------------+
|     google|  1265576301|
|       NULL|   678390825|
|   facebook|   294191864|
|     kenh14|    46361874|
|       soha|    31286779|
|      cafef|    30089621|
|news.google|    27616243|
|     coccoc|     9445743|
|taboolanews|     8075838|
|    afamily|     7493689|
+-----------+------------+
only showing top 10 rows



In [14]:
# Import dữ liệu bảng website view by cate
query = """(SELECT 
            date, parent_cat, type, site, views
            FROM  website_view_by_cat) 
            AS subquery """
df_cat= spark.read.jdbc(url=url, table=query, properties=properties)

df_cat.show()


+-------------------+--------------------+------+-----+------+
|               date|          parent_cat|  type| site| views|
+-------------------+--------------------+------+-----+------+
|2021-11-01 00:00:00|doanh-nghiep-gioi...|mobile|cafef|  2333|
|2021-11-01 00:00:00|            tien-ich|mobile|cafef|     1|
|2021-11-01 00:00:00|               upcom|    pc|cafef|    12|
|2021-11-01 00:00:00|               other|mobile|cafef| 35950|
|2021-11-01 00:00:00|        bat-dong-san|mobile|cafef|284065|
|2021-11-01 00:00:00|MBS-cong-ty-co-ph...|mobile|cafef|     8|
|2021-11-01 00:00:00|CTS-cong-ty-co-ph...|mobile|cafef|     1|
|2021-11-01 00:00:00|                hose|    pc|cafef|    38|
|2021-11-01 00:00:00|                home|    pc|cafef|342049|
|2021-11-01 00:00:00|du-lieu-doanh-nghiep|mobile|cafef|     7|
|2021-11-01 00:00:00|        doanh-nghiep|    pc|cafef| 48085|
|2021-11-01 00:00:00|nghe-tai-chinh-ng...|    pc|cafef|     1|
|2021-11-01 00:00:00|            thi-truo|    pc|cafef|

In [15]:
#Số lượng views theo parent_cat

from pyspark.sql.functions import *
group_cat=df_cat.groupBy('date','parent_cat').agg(sum('views').alias('total_views')).orderBy(['date','total_views'],ascending=[True,False])
group_cat.show()
                                                                                             

+-------------------+--------------------+-----------+
|               date|          parent_cat|total_views|
+-------------------+--------------------+-----------+
|2021-09-30 00:00:00|                home|     418769|
|2021-09-30 00:00:00|                NULL|     284950|
|2021-09-30 00:00:00|          hau-truong|     274695|
|2021-09-30 00:00:00|            giai-tri|     242285|
|2021-09-30 00:00:00|                song|     190116|
|2021-09-30 00:00:00|           lifestyle|     160498|
|2021-09-30 00:00:00|            giao-duc|     140498|
|2021-09-30 00:00:00|cau-chuyen-kinh-d...|     138310|
|2021-09-30 00:00:00|                 dep|     128197|
|2021-09-30 00:00:00|            suc-khoe|     116223|
|2021-09-30 00:00:00|                quiz|     112254|
|2021-09-30 00:00:00|              tam-su|     112225|
|2021-09-30 00:00:00|           tieu-dung|      97917|
|2021-09-30 00:00:00|            the-gioi|      91797|
|2021-09-30 00:00:00|                 yeu|      81218|
|2021-09-3

In [16]:
# Import dữ liệu bảng website view by source
query = """(SELECT 
           date, type, refer, site, newsid, views, new_view
            FROM  website_view_by_source) 
            AS subquery """
df_view_source= spark.read.jdbc(url=url, table=query, properties=properties)

df_view_source.show()


+-------------------+------+------+----+-----------------+-----+--------+
|               date|  type| refer|site|           newsid|views|new_view|
+-------------------+------+------+----+-----------------+-----+--------+
|2024-07-23 00:00:00|mobile|  NULL|genk|20240722115150654|   23|    NULL|
|2024-07-23 00:00:00|mobile|  genk|genk|20240513095410082|    4|       4|
|2024-07-23 00:00:00|mobile|  genk|genk|20240628080238254|    6|       1|
|2024-07-23 00:00:00|mobile|google|genk| 2011022806029243|    9|       3|
|2024-07-23 00:00:00|mobile|google|genk|20141020111347011|    3|       3|
|2024-07-23 00:00:00|mobile|google|genk| 2021042616032764|    6|       5|
|2024-07-23 00:00:00|mobile|google|genk| 2024010811061134|    7|    NULL|
|2024-07-23 00:00:00|mobile|google|genk|20240326100800122|    4|       2|
|2024-07-23 00:00:00|mobile|google|genk|20240629101901836|  154|      21|
|2024-07-23 00:00:00|mobile|google|genk|20240709214621586|    9|       2|
|2024-07-23 00:00:00|    pc|  NULL|gen

In [17]:
#Top 10 traffice source thu hút nhiều user mới nhất

from pyspark.sql.functions import *
group_view_source=df_view_source.groupBy('refer').agg(sum('new_view').alias('total_new_access')).orderBy('total_new_access',ascending=False)
group_view_source.show(10)

+--------+----------------+
|   refer|total_new_access|
+--------+----------------+
|  google|        23422403|
|  kenh14|        15172361|
|facebook|         8663988|
|    soha|         4251452|
|   cafef|         3102840|
| afamily|         1685151|
| cafebiz|          676174|
|    genk|          552192|
|   gamek|          409922|
|    news|          206397|
+--------+----------------+
only showing top 10 rows



In [20]:
#Merge 2 bảng info và view by source

from pyspark.sql.functions import *

merged_info_view_source = df_info.join(df_view_source, (df_info.newsid == df_view_source.newsid) & (df_info.sourceNews == df_view_source.site),how='right')


In [27]:
#Số lượng view theo giờ publish của các bài viết từ FB,GG

from pyspark.sql.functions import *
filter_merged = merged_info_view_source.filter((col('refer').isin('google','facebool')) & (col('url').isNotNull()))
group_filter=filter_merged.groupBy('refer','publish_hour').agg(sum('views').alias('total_views')).orderBy('publish_hour',ascending=True)
group_filter.show()

+------+------------+-----------+
| refer|publish_hour|total_views|
+------+------------+-----------+
|google|           0|    3198524|
|google|           1|     280188|
|google|           2|      47917|
|google|           3|      68246|
|google|           4|      67332|
|google|           5|     719724|
|google|           6|    3990988|
|google|           7|    6986715|
|google|           8|    4778501|
|google|           9|    5920420|
|google|          10|    7072031|
|google|          11|    5743036|
|google|          12|    2960246|
|google|          13|    4551966|
|google|          14|    7289701|
|google|          15|    6343577|
|google|          16|    4886612|
|google|          17|    3607048|
|google|          18|    2897955|
|google|          19|    5774926|
+------+------------+-----------+
only showing top 20 rows



In [30]:
#Top 10 bài viết thu hút nhiều user nhất từ Google

from pyspark.sql.functions import *

filter_merged = merged_info_view_source.filter(col('refer').isin('google') & col('url').isNotNull())
group_merged=filter_merged.groupBy('title','publish_date','url').agg(sum('views').alias('total_views')).orderBy('total_views',ascending=False)
group_merged.show(10)

+--------------------+-------------------+--------------------+-----------+
|               title|       publish_date|                 url|total_views|
+--------------------+-------------------+--------------------+-----------+
|Truy tìm Trần Khá...|2024-07-31 00:00:00|https://cafef.vn/...|    1136044|
|Công bố 12 số điệ...|2024-07-29 00:00:00|https://cafef.vn/...|     747857|
|Một nhân vật tron...|2024-08-08 00:00:00|https://cafef.vn/...|     575903|
|Nam sinh học hết ...|2024-07-22 00:00:00|https://cafef.vn/...|     534382|
|Xuất hiện quả mít...|2024-08-07 00:00:00|https://kenh14.vn...|     516419|
|Ông Trịnh Văn Quy...|2024-07-23 00:00:00|https://cafef.vn/...|     506687|
|1 mình chăm mẹ, k...|2024-08-01 00:00:00|https://kenh14.vn...|     442109|
|Bắt Giám đốc Nguy...|2024-08-03 00:00:00|http://soha.vn/ba...|     403459|
|Chê tiền đền bù í...|2024-08-04 00:00:00|https://kenh14.vn...|     387577|
|Tổng bí thư, Chủ ...|2024-08-18 00:00:00|https://cafef.vn/...|     378718|
+-----------